In [5]:
# Imports

import pandas as pd

from tinydb import TinyDB, Query
from tinydb.storages import JSONStorage
from tinydb.middlewares import CachingMiddleware

from tqdm import tqdm_notebook as tqdm

In [17]:
# Column names

symbol_df = pd.DataFrame(columns=['Symbol', 'Name', 'Symbol2', 'Currency', 'Exchange', 'Status'])

In [18]:
# Exchanges

exchanges = {
    'FWB': 'ib_fwb_etfs.csv',
    'LSE': 'ib_lse_etfs.csv',
    'LSEETF': 'ib_lse-etf_etfs.csv',
    'IBIS': 'ib_xetra_etfs.csv'
}

In [19]:
# Read symbols info from file to df

for ex, file in exchanges.items():
    one_ex_data = pd.read_csv(file, delimiter=';')
    # todo: clean symbol names
    one_ex_data['Exchange'] = ex
    symbol_df = symbol_df.append(one_ex_data, ignore_index=True, sort=True)
symbol_df.describe()

,Currency,Exchange,Name,Status,Symbol,Symbol2
count,5323,5323,5323,0,5323,5323
unique,3,4,2284,0,3732,3955
top,EUR,LSE,WisdomTree US Equity Income UCITS ETF,NaN,STAW,3DES
freq,3024,1827,12,NaN,4,3


In [10]:
# Write df to CSV

symbol_df.to_csv('symbol_database.csv', sep=';')

In [23]:
# Write list to textfile

# df = symbol_database[symbol_database['Symbol']!=symbol_database['Symbol2']]
df = symbol_df.loc[symbol_df['exchange']=='fwb', 'Symbol']
symbol_list = df.tolist()

file = open('symbollist.txt', 'w+')
for symbol in symbol_list:
    file.write(symbol+'\n')
file.close()

In [20]:
# Write to tinydb

# ----- DANGER ZONE! -----
# 
# symbol_db = TinyDB('symbol_db.json', storage=CachingMiddleware(JSONStorage))
# 
# for index, row in tqdm(symbol_df.iterrows()):
#     symbol_db.insert({'Id': index, 
#                       'Symbol': row['Symbol'], 
#                       'Name': row['Name'], 
#                       'Currency': row['Currency'], 
#                       'Exchange': row['Exchange'],
#                       'Status': 'untouched'
#                      })
# symbol_db.close()

In [6]:
# Read symbol data to df

symbol_db = TinyDB('symbol_db.json')
data_list = symbol_db.all()
symbol_db.close()

data_string = str(data_list)
data_string = data_string.replace("-'bP'", "-bP")
data_string = data_string.replace("MOODY'S", "MOODYS")
data_string = data_string.replace("'", '"')

df = pd.read_json(data_string, orient='records')
df

,Id,Symbol,Name,Currency,Exchange,Status
0,0,BNQF,Collateralized ETC on RICI Enhanced Gas Oil TR...,EUR,FWB,Market data farm connection is OK:usfarm
1,1,CNB,Lyxor Euro Corporate Bond Ex Financials UCITS ETF,EUR,FWB,Finished
2,2,UEF5,UBS ETF - MSCI Emerging Markets Socially Respo...,EUR,FWB,Historical Market Data Service error message:N...
3,3,GOMA,BNP Paribas Easy Barclays Euro Government Infl...,EUR,FWB,Historical Market Data Service error message:N...
4,4,X0BM,Coba ETC -2 x Platinum Daily Short Index,EUR,FWB,Finished
5,5,FLOT,Lyxor Barclays Floating Rate Euro 0-7Y UCITS ETF,EUR,FWB,Historical Market Data Service error message:N...
6,6,X0E2,Coba ETC 2x Gasoil Daily Long Index,EUR,FWB,Finished
7,7,ETLH,L&G Ecommerce Logistics UCITS ETF,EUR,FWB,Historical Market Data Service error message:N...
8,8,PP2,ComStage ETF PSI 20 Leverage UCITS ETF,EUR,FWB,Historical Market Data Service error message:N...
9,9,XBO3,Xtrackers II Italy Government Aggregate Bond S...,EUR,FWB,Historical Market Data Service error message:N...


In [8]:
# Group by exchange

dfe = df.groupby('Exchange').count()
dfe

,Id,Symbol,Name,Currency,Status
Exchange,,,,,
FWB,1349,1349,1349,1349,1349
IBIS,1266,1266,1266,1266,1266
LSE,1827,1827,1827,1827,1827
LSEETF,881,881,881,881,881


In [9]:
# Group by status

dfs = df.groupby('Status').count()
dfs

,Id,Symbol,Name,Currency,Exchange
Status,,,,,
Finished,3079,3079,3079,3079,3079
Historical Market Data Service error message:HMDS query returned no data: 0A12@LSE Midpoint,1,1,1,1,1
Historical Market Data Service error message:HMDS query returned no data: 0A1I@LSE Midpoint,1,1,1,1,1
Historical Market Data Service error message:HMDS query returned no data: 0A1T@LSE Midpoint,1,1,1,1,1
Historical Market Data Service error message:HMDS query returned no data: 0AND@LSE Midpoint,1,1,1,1,1
Historical Market Data Service error message:HMDS query returned no data: 0ANE@LSE Midpoint,1,1,1,1,1
Historical Market Data Service error message:HMDS query returned no data: 0ANH@LSE Midpoint,1,1,1,1,1
Historical Market Data Service error message:HMDS query returned no data: 0ANI@LSE Midpoint,1,1,1,1,1
Historical Market Data Service error message:HMDS query returned no data: 0ANK@LSE Midpoint,1,1,1,1,1


In [7]:
# Group by status + exchange

dfse = df.groupby(['Exchange', 'Status']).count()
dfse

Id  Symbol  \
Exchange Status                                                            
FWB      Finished                                            375     375   
         Historical Market Data Service error message:No...  962     962   
         Historical Market Data Service error message:No...    2       2   
         Market data farm connection is OK:usfarm              1       1   
         No security definition has been found for the r...    9       9   
...                                                          ...     ...   
LSEETF   Historical Market Data Service error message:HM...    1       1   
         Historical Market Data Service error message:No...    1       1   
         Historical Market Data Service error message:No...    1       1   
         Historical Market Data Service error message:No...    2       2   
         Historical Market Data Service error message:No...    1       1   

                                                             Name  Currency  
Exchange Status                                                              
FWB      Finished                                             375       375  
         Historical Market Data Service error message:No...   962       962  
         Historical Market Data Service error message:No...     2         2  
         Market data farm connection is OK:usfarm               1         1  
         No security definition has been found for the r...     9         9  
...                                                           ...       ...  
LSEETF   Historical Market Data Service error message:HM...     1         1  
         Historical Market Data Service error message:No...     1         1  
         Historical Market Data Service error message:No...     1         1  
         Historical Market Data Service error message:No...     2         2  
         Historical Market Data Service error message:No...     1         1  

[364 rows x 4 columns]

In [30]:
# Update certain status values

symbol_db = TinyDB('symbol_db.json')

my_query = Query()
# symbol_db.update({'Status': 'untouched'}, 
#                  my_query.Status == "Historical Market Data Service error message:No market data permissions for SWB STK")

symbol_db.close()

In [11]:
df.loc[(df['Exchange']=='LSE') & (df['Status']!='Finished')]

,Id,Symbol,Name,Currency,Exchange,Status
1354,1354,4GLD,Xetra-Gold,EUR,LSE,Historical Market Data Service error message:H...
1358,1358,0I9M,Direxion Daily Retail Bull 3X Shares,USD,LSE,Historical Market Data Service error message:H...
1361,1361,EQUA,BNP PARIBAS EASY Equity Quality Europe UCITS ETF,EUR,LSE,Historical Market Data Service error message:H...
1369,1369,JBGOUA,GAM Precious Metals - Physical Gold,USD,LSE,Historical Market Data Service error message:H...
1373,1373,E903,ComStage 1 DivDAX UCITS ETF,EUR,LSE,Historical Market Data Service error message:H...
1375,1375,OAUE,UBS ETF CH-Gold USD,EUR,LSE,Historical Market Data Service error message:H...
1376,1376,EL4S,Deka DB EUROGOV Germany 1-3 UCITS ETF,EUR,LSE,Historical Market Data Service error message:H...
1379,1379,C054,ComStage ETF FR EURO STOXX 50 UCITS ETF,EUR,LSE,Historical Market Data Service error message:H...
1380,1380,USMVD,OSSIAM US MINIMUM VARIANCE ESG NR UCITS ETF,USD,LSE,Historical Market Data Service error message:H...
1382,1382,0L0S,SPDR S&P Aerospace & Defense ETF,USD,LSE,Historical Market Data Service error message:H...
